# Topic 8 Laboratory

In this activity, you will use imbalanced datasets from the [Keel](https://sci2s.ugr.es/keel/imbalanced.php) public repository to appropriately evaluate performance metric in two popular classifiers.

## Performance Evaluation in Binary Datasets

### Downloading and making sense of the necessary data

We will use the [glass](https://sci2s.ugr.es/keel/dataset/data/imbalanced/glass-names.txt) dataset. This one has 214 samples of 7 different types of glasses. Each glass sample has 9 features, each corresponding to a different element that composes a glass sample: Rl(??), Na, Mg, Al, Si, K, Ca, Ba and Fe.

Since we will start with binary datasets, we will first use the [glass0](https://sci2s.ugr.es/keel/keel-dataset/datasets/imbalanced/imb_IRlowerThan9/names/glass0-names.txt) version of the dataset. The only difference of this one with respect of the original one is that a certain glass class called `class 0` is compared against the rest of the glass classes.

Click [here](https://sci2s.ugr.es/keel/dataset.php?cod=141) to visit the *glass0* description website, go to the bottom (where it says **Files and additional references** and download the **complete data set**. Unzip the file and store it **in the same directory as this jupyter notebook**. You should get a file with the extension *.dat*. OR, you can simply download `glass0.dat` from Moodle!

### Loading the Data

Now we will import the data from the file into Python, to do so, run the following cell:

In [ ]:
import numpy as np

data = np.genfromtxt('glass0.dat',
                     usecols=range(9), # this only brings the first nine columns of the dataset
                     skip_header=14, # The first 14 lines of the .dat contain a description
                     delimiter=',')


target_names = np.genfromtxt('glass0.dat',
                     usecols=range(9,10), # This brings the last column of the dataset, which has the class
                     dtype = None,
                     encoding = None, # This helps us get the strings in a numpy array
                     skip_header=14,
                     delimiter=',')

print(data,data.shape)
print(target_names,target_names.shape)

Notice that once again, we will store the data and the target in separate variables. Moreover, we will generate a new target variable called `target` with `positive=0` and `negative=1` using the **list comprehension** technique:

In [ ]:
target = []
target = [0 if i == ' positive' else 1 for i in target_names]
target=np.array(target)
print(target,target.shape)

Notice that we chose 0 to be the positive and 1 to be the negative since the [glass0 documentation](https://sci2s.ugr.es/keel/dataset.php?cod=141) said so!

### Training the Classifiers

Now we need to train a supervised learning model to evaluate. In this case we will use two popular classification models so that we can compare which is better for this dataset: **Support Vector Machine (SVM)** vs. **Random Forests (RF)**:

First we need to split our dataset into training and testing sets (80% train, 20% test). You have already done this in the past!

In [ ]:
## Use this cell to split your dataset into training and testing w/ stratification


Now train a SVM model called `model_svm` using the training data and predict the test data. Store the fitted model in a variable called `clf_svm` and the prediction results in a variable called `y_svm`.

In [ ]:
## Use this cell to train a SVM model to predict the labels of the test data


Likewise, train a RF model called `model_rf` (using the `RandomForestClassifier()` function contained in `sklearn.ensemble` package) on the same training data and predict the test data. Store the results in variables called `clf_rf` and `y_rf`.

In [ ]:
## Use this cell to train a RF model to predict the labels of the test data


Both `y_svm` and `y_rf` should be *numpy* array vectors of size (43,)

### Performance Evaluation Metrics

In week 3 lab, we saw that once that we predict the values for the test set, we can compare both the `y_test` vector with the predicted vectors (in our case `y_svm` and `y_rf`) to obtain the results. If you recall, it was quite tricky to do this as we had more classes in the vector! Therefore, this time we will do it slightly different...

#### Accuracy

By now you should know that everything in Python can be imported! Performance metrics can be imported as well. We will first use accuracy (although it should be very clear by now that this one is not suitable for imbalanced datasets).

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy of SVM: ', accuracy_score(y_svm, y_test))
print('Accuracy of RF: ', accuracy_score(y_rf, y_test))

**Which is the best classifier in terms of accuracy?**

**Answer:**

#### AUC-ROC

In this week's lecture we saw that one of the most robust metrics for accuracy in the *imbalanced* world is the area under the receiver operating characteristic curve (or AUC-ROC for short). To get this metric, we need to retrain our models, this time in a probabilistic way!

This means that by setting the input `probability=True` when generating our SVM model (RF doesn't need it) and by changing `predict` to `predict_proba`, we will not obtain a fixed output, but instead a probability (value between 0 and 1) of the classification to be 0 or 1 respectively.

To keep the previous models/classes/predictions in our workspace, train new (probabilistic) models in the following cell and store the trained models as `model_svm_proba` and `model_rf_proba`, the classifiers as `clf_svm_proba` and `clf_rf_proba`, and the outputs as `y_svm_proba` and `y_rf_proba`:

In [ ]:
## Use this cell to re-train the models in a probabilistic way


For `y_svm_proba` and `y_rf_proba` you should get *numpy* arrays of shape (43, 2), as now each output contains the probability of each sample to be 0 (first value) or 1 (second value).

Now we will evaluate the AUC-ROC. We will **not** get the plot (as we don't have different thresholds), but rather we will obtain the numeric area under the curve (a value between 0 and 1, the larger the better).

To obtain this values, you can run the following cell (provided that you have correctly calculated `y_svm_proba` and `y_rf_proba` in the previous step):

In [ ]:
## Testing AUC-ROC considering the probabilities of the positive class (i.e. 0)
from sklearn.metrics import roc_auc_score

# First, we need to extract only the probabilities of classifying 0 (second column of our numpy arrays)
# We can do this once again by means of list comprehension
y_svm_proba_0 = [p[0] for p in y_svm_proba]
y_rf_proba_0 = [p[0] for p in y_rf_proba]

print('AUC for SVM: ',roc_auc_score(y_test, y_svm_proba_0))
print('AUC for RF: ',roc_auc_score(y_test, y_rf_proba_0))

If you are getting AUC-ROC values **smaller than 0.5** then you can simply invert them (or get the AUC-ROC for the rest/negative class) to get the actual AUC-ROC value for this classifier.

A comprehensive explanation of why AUC-ROC cannot be smaller than 0.5 can be found [here](https://www.datascienceblog.net/post/machine-learning/interpreting-roc-curves-auc/).

In [ ]:
## Use this cell to test AUC-ROC considering the probabilities of the rest/negative class (i.e. 1)


**Is the classifier that was better in accuracy still better in AUC?**

**ANSWER:**

### Cross Validation

Let's see if the previous results were not product of chance or a lucky data split! To do so, we will use cross validation with $k=5$ folds.

Using the `KFold()`function contained in the `sklearn.model_selection` package you can split a dataset (in this case, the original one contained in the `data` variable) by using the `get_n_splits()` method 

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(data)
print(kf)

So far you only get a `KFold` object. However, we can put this object inside a for loop to create different training and testing sets in each iteration:

In [ ]:
i=1
for train_index, test_index in kf.split(data):
    print('\nFold '+str(i)+':')
    print('TRAIN INDEXES:', train_index)
    print('TEST INDEXES:', test_index)
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    i+=1

Notice that what is being printed in the cell above are not the samples, but the indexes from where the samples will be taken in each fold!

Now use the cell below to implement the **stratified** version of KFolds. Remember that stratification will ensure that both classes are equally present in each set for every fold. 

In [ ]:
## Use this cell to do a KFolds split in a stratified way
# Hint: Use StratifiedKFold


To evaluate models using cross validation you **don't** need all of this! In fact, you only use this if you want to store your folds or if you want to implement more complex models.

As a matter of fact it is easier to simply import the `cross_validate` function from the `sklearn.model_selection` package and get the scores as follows:

In [ ]:
# Cross validatig the original data (the function will do it all)
from sklearn.model_selection import cross_validate

scores_svm = cross_validate(model_svm, data, target, cv=5)
print('SVM cross-validated scores: ', scores_svm)
scores_rf = cross_validate(model_rf, data, target, cv=5)
print('RF cross-validated scores: ', scores_rf)

**From these metrics do you notice any disadvantage of RF?**

**Answer:**

In fact, these are not the only metrics you can get from cross validation! Using the `cross_val_scores` function from the `sklearn.model_selection` package you can get (almost) all of the measures that we discussed in class. You can check the [scoring documentation](https://scikit-learn.org/stable/modules/model_evaluation.html) to see what you can calculate,and below you will find some examples

In [ ]:
# First we import cross_val_scores
from sklearn.model_selection import cross_val_score

In [ ]:
# Evaluating Accuracy for each fold
print('Accuracy for SVM: ',cross_val_score(model_svm, data, target, cv=5, scoring = 'accuracy'))
print('Accuracy for RF: ',cross_val_score(model_rf, data, target, cv=5, scoring = 'accuracy'))

In [ ]:
# Evaluating Mean Accuracy for all folds
print('Mean Accuracy for SVM: ',np.mean(cross_val_score(model_svm, data, target, cv=5, scoring = 'accuracy')))
print('Mean Accuracy for RF: ',np.mean(cross_val_score(model_rf, data, target, cv=5, scoring = 'accuracy')))

In [ ]:
# Evaluating Precision, Recall and F1-score (both by fold and average)
print('Precision for SVM: ',cross_val_score(model_svm, data, target, cv=5, scoring = 'precision'))
print('Precision for RF: ',cross_val_score(model_rf, data, target, cv=5, scoring = 'precision'))
print('Recall for SVM: ',cross_val_score(model_svm, data, target, cv=5, scoring = 'recall'))
print('Recall for RF: ',cross_val_score(model_rf, data, target, cv=5, scoring = 'recall'))
print('F1-Score for SVM: ',cross_val_score(model_svm, data, target, cv=5, scoring = 'f1'))
print('F1-Score for RF: ',cross_val_score(model_rf, data, target, cv=5, scoring = 'f1'))

print('Mean Precision for SVM: ',np.mean(cross_val_score(model_svm, data, target, cv=5, scoring = 'precision')))
print('Mean Precision for RF: ',np.mean(cross_val_score(model_rf, data, target, cv=5, scoring = 'precision')))
print('Mean Recall for SVM: ',np.mean(cross_val_score(model_svm, data, target, cv=5, scoring = 'recall')))
print('Mean Recall for RF: ',np.mean(cross_val_score(model_rf, data, target, cv=5, scoring = 'recall')))
print('Mean F1-Score for SVM: ',np.mean(cross_val_score(model_svm, data, target, cv=5, scoring = 'f1')))
print('Mean F1-Score for RF: ',np.mean(cross_val_score(model_rf, data, target, cv=5, scoring = 'f1')))

Notice that in the Keel repository, there is already a version of the glass dataset called **5FCV**. This means that the authors of this repository already provide a "proper" partition of this dataset to test cross validation. If you download this version, you will get 10 *.dat* files instead of 1. **Do you know why?**

**Answer:**

## Bonus
Run the tests using the 5FCV version of the dataset and compare the results